In [1]:
import numpy as np
from utils import *
from pathlib import Path
import time

In [2]:
# Hyperparamters
d = 2       # Dimensionality
s = 2       # Number of discrete value for each dimension
trials = 4  # Number of trials per signal length
n = 19      # 2^{n} is the max length of trial signal

# Discrete means set
angles = np.linspace(0, 2 * np.pi, s, endpoint=False)
grids = np.meshgrid(*([angles] * d), indexing='ij')
Theta = np.stack(grids, axis=-1).reshape(-1, d)

In [3]:
# setting up recorded csv
output_path = Path(f"benchmark_results/time_vs_samples_{d}D_{s}MeansGrid.csv")

# Write CSV header once
if not output_path.exists():
    with open(output_path, 'w') as f:
        f.write("signal_length,mean_exe_time_sec,std_exe_time_sec\n")

for i in range(1, n+1):
    signal_length = 2 ** i
    exe_times = np.empty(trials, dtype=np.float64)
    for j in range(trials):
        # Generate random signal
        signal = np.column_stack([
            np.random.uniform(0, 2 * np.pi, signal_length)
            for _ in range(d)
        ])

        # Time the opart function
        start = time.perf_counter()
        chpnts, signal_mean = opart(signal, Theta, i)
        end = time.perf_counter()

        exe_times[j] = end - start

    # Compute stats
    mean_time = np.mean(exe_times)
    std_time  = np.std(exe_times)

    # Record to CSV
    with open(output_path, 'a') as f:
        f.write(f"{signal_length},{mean_time:.6f},{std_time:.6f}\n")